<a href="https://colab.research.google.com/github/jongbokhi/Time-Series_model/blob/master/prediction_exchange_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import yfinance as yf
from fredapi import Fred

import pandas_datareader as pdr

In [181]:
fred_key = '88470199715d373130aa3fb38170ed0c'
fred = Fred(api_key=fred_key)

In [ ]:
### Prepare for Dataset
 ## Source: FRED ECONOMIC DATA, YAHOO FINANCE 
 ## TIME PERIOD: [2010-7-1 ~ 2023-3-1]

In [182]:
if __name__ == '__main__':
    #  Set start date
    start_date = datetime.date(2010, 7, 1)
    start_date_str = datetime.datetime.strftime(start_date, "%Y-%m-%d")

    #  Federal Reserve Economic Data Service
    data_source1 = 'fred'
    data_source2 = 'yf'
    usd_kwon = 'DEXKOUS'
    federal_funds_effective_rate = 'FEDFUNDS'
    global_price_brent_crude_oil= 'POILBREUSDM'
    global_price_of_wti_crude_oil = 'POILWTIUSDM'
    nasdaq100_Index = 'NASDAQ100'


    #  Fetch data from FED 
    usd_kwon_df = pdr.DataReader(usd_kwon, data_source1, start = start_date, end='2023 - 03 - 01')
    effective_rate_df = pdr.DataReader(federal_funds_effective_rate, data_source1, start = start_date, end='2023 - 03 - 01')
    wti_crude_oil_df = pdr.DataReader(global_price_brent_crude_oil, data_source1, start = start_date, end='2023 - 03 - 01')
    crude_oil_df = pdr.DataReader(global_price_of_wti_crude_oil, data_source1, start = start_date, end='2023 - 03 - 01')
    
    

In [183]:
#load data from Yahoo Finance

tesla = yf.Ticker("TSLA")

# get historical market data
hist = tesla.history(start= '2010-07-01', end= '2023-03-02')
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0
2010-07-07 00:00:00-04:00,1.093333,1.108667,0.998667,1.053333,103825500,0.0,0.0
2010-07-08 00:00:00-04:00,1.076000,1.168000,1.038000,1.164000,115671000,0.0,0.0


In [184]:
tesla_open = hist['Open']
tesla_open.head(5)

Date
2010-07-01 00:00:00-04:00    1.666667
2010-07-02 00:00:00-04:00    1.533333
2010-07-06 00:00:00-04:00    1.333333
2010-07-07 00:00:00-04:00    1.093333
2010-07-08 00:00:00-04:00    1.076000
Name: Open, dtype: float64

In [185]:
# unifiy timezone of timeindex for merging dataframes

tesla_open.index = tesla_open.index.tz_localize(None)

In [186]:
#change the name of timeindex
tesla_open = tesla_open.rename_axis('DATE')
tesla_open.tail()

DATE
2023-02-23    203.910004
2023-02-24    196.330002
2023-02-27    202.029999
2023-02-28    210.589996
2023-03-01    206.210007
Name: Open, dtype: float64

In [187]:
# Merge data frames
merged_df = pd.merge(effective_rate_df, usd_kwon_df, on='DATE', how='left')

merged_df = pd.merge(merged_df, wti_crude_oil_df, on='DATE', how='left')

merged_df = pd.merge(merged_df, crude_oil_df, on='DATE', how='left')

merged_df = pd.merge(merged_df, tesla_open, on='DATE', how='left')

In [188]:
# change column name in merged dataframe

df = merged_df.rename(columns={'FEDFUNDS': 'effective_rate', 'DEXKOUS': 'usd_kwon', 'POILBREUSDM': 'brent_crude_oil', 'POILWTIUSDM' :'wti_crude_oil', 'Open' : 'tesla_open_price' })

df.tail()

,effective_rate,usd_kwon,brent_crude_oil,wti_crude_oil,tesla_open_price
DATE,,,,,
2022-11-01,3.78,1416.20,90.938636,84.076364,234.050003
2022-12-01,4.10,1299.11,81.503182,76.581818,197.080002
2023-01-01,4.33,NaN,84.081818,78.251364,NaN
2023-02-01,4.57,1231.77,83.632500,77.027500,173.889999
2023-03-01,4.65,1315.15,79.256522,73.348261,206.210007


In [189]:
### Target Variable : usd_kwon (USD/KRW)
### predictors: effective_rate, brent_crude_oil, wti_crude_oil, tesla_open_price